<a href="https://colab.research.google.com/github/Luke-zm/coursera_learning/blob/main/udemy_pyomo/pyomo_lp_tut2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Tutorial notes on udemy course.
!pip install pyomo
!apt-get install -y -qq glpk-utils

Decison variables:
$$
\\x_1: num~ ~desk
\\x_2: num~ ~table
\\x_3: num~ ~chairs
$$
Obj
$$
\max(z) = 60 x_1 + 30 x_2 + 20 x_3
$$

$$
s.t
\\8x_1 + 6x_2 + x_3 \leq 48
\\4x_1 + 2x_2 + 1.5 x_3 \leq 20
\\2x_1 + 1.5x_2 + 0.5x_3 \leq 8
\\ x_2 \leq 5
$$


When the number of decision vraibles increase, then this way of writing out decison var 1 by 1 will not work.
The more professional way is use a range of numbers/ sets of numbers to make formulation easier for myself.
$$
\mathcal{x}_i,~ ~where~i = \{1,2,3\}
\\max(z) = \sum_{i=1}^{n} P_i \times x_i
\\s.t
\\ \sum_{i=1}^{n} L_i \times x_i \leq 48
\\ \sum_{i=1}^{n} F_i \times x_i \leq 20
\\ \sum_{i=1}^{n} C_i \times x_i \leq 8
\\ x_2 \leq 5
$$

Always formulate the problem using sets

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [ ]:
model = pyo.ConcreteModel()

# Sets, usually work as index
model.i = pyo.Set(initialize = ['Desk', 'Table', 'Chair'])

# Parameters
model.L = pyo.Param(model.i, initialize = {'Desk':8, 'Table': 6, 'Chair': 1})
L = model.L
model.F = pyo.Param(model.i, initialize = {'Desk':4, 'Table': 2, 'Chair': 1.5})
F = model.F
model.C = pyo.Param(model.i, initialize = {'Desk':2, 'Table': 1.5, 'Chair': 0.5})
C = model.C
model.Price = pyo.Param(model.i, initialize = {'Desk':60, 'Table': 30, 'Chair': 20})
Price = model.Price

In [ ]:
# Decision variable
# Anytime in the model formulation there's index, then inclue the index.
model.x = pyo.Var(model.i, within=pyo.NonNegativeReals)
x = model.x

In [ ]:
# Objective function
# because of using a pyo.Set(), better to define a function for Obj and Constraint
def objective_rule(model):
  return sum(Price[i]*x[i] for i in model.i)

# Pass this function to pyomo
model.Obj = pyo.Objective(rule = objective_rule, sense = pyo.maximize)

In [ ]:
# Define the constaint functions
# Note: don't really need to define i again over here, but best practice is just put here
def lumber_cons(model,i):
  return sum(L[i]*x[i] for i in model.i)<=48
model.lumber_con = pyo.Constraint(model.i, rule=lumber_cons)

def fin_cons(model, i):
  return sum(F[i]*x[i] for i in model.i) <=20
model.fin_con = pyo.Constraint(model.i, rule=fin_cons)

def cap_cons(model, i):
  return sum(C[i]*x[i] for i in model.i) <=8
model.cap_con = pyo.Constraint(model.i, rule=cap_cons)

# The last constriant
def tab_num_con(model, i):
  return x['Table'] <=5
model.tab_num_con = pyo.Constraint(model.i, rule=tab_num_con)

In [ ]:
# Solve
Solver = SolverFactory('glpk')

results = Solver.solve(model)

print(results)
print('Objective function = ', model.Obj())

# x is not unique, hence need to call pyomo set used for indexing to find the correct element
for i in model.i:
  print('Number of', i, 'produced = ', x[i]())


Problem: 
- Name: unknown
  Lower bound: 280.0
  Upper bound: 280.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 3
  Number of nonzeros: 30
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0037021636962890625
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective function =  280.0
Number of Desk produced =  2.0
Number of Table produced =  0.0
Number of Chair produced =  8.0
